In [ ]:
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))

In [1]:
import numpy as np
import torch

In [2]:
def calculate_edges1(nodes):
    return torch.abs(nodes.unsqueeze(1) - nodes).flatten(end_dim=-2)

In [3]:
def calculate_edges2(nodes):
    n_nodes = nodes.shape[2]
    r, c = np.triu_indices(n_nodes, 1)
    return np.abs(
        nodes[:, :, r] - nodes[:, :, c]
    )  # / (nodes[:, :, r] + nodes[:, :, c])

In [4]:
nodes = torch.rand(34, 4)
edges = calculate_edges1(nodes).reshape(34, 34, 4)
edges.shape

torch.Size([34, 34, 4])

In [5]:
def anti_vectorize(vector: np.ndarray, n_nodes: int) -> np.ndarray:
    adj_matrix = np.zeros((n_nodes, n_nodes))
    adj_matrix[np.tril_indices(n_nodes, k=-1)] = vector
    adj_matrix = adj_matrix.transpose()
    adj_matrix[np.tril_indices(n_nodes, k=-1)] = vector
    return adj_matrix

In [6]:
nodes_more = torch.stack([nodes, nodes]).permute(2, 0, 1)
edges2 = calculate_edges2(nodes_more)
edges_only = edges2[0, 0]
antivectorized = anti_vectorize(edges_only, 34)

In [7]:
antivectorized

array([[0.        , 0.38950217, 0.29722285, ..., 0.3489905 , 0.08027154,
        0.13640803],
       [0.38950217, 0.        , 0.17587149, ..., 0.83714145, 0.60738117,
        0.62455899],
       [0.29722285, 0.17587149, 0.        , ..., 0.08748412, 0.63090205,
        0.30006659],
       ...,
       [0.3489905 , 0.83714145, 0.08748412, ..., 0.        , 0.48091918,
        0.22871214],
       [0.08027154, 0.60738117, 0.63090205, ..., 0.48091918, 0.        ,
        0.08262956],
       [0.13640803, 0.62455899, 0.30006659, ..., 0.22871214, 0.08262956,
        0.        ]])

In [8]:
edges[:, :, 0]

tensor([[0.0000, 0.3895, 0.2972,  ..., 0.3723, 0.0610, 0.1436],
        [0.3895, 0.0000, 0.0923,  ..., 0.7618, 0.4505, 0.5331],
        [0.2972, 0.0923, 0.0000,  ..., 0.6695, 0.3582, 0.4408],
        ...,
        [0.3723, 0.7618, 0.6695,  ..., 0.0000, 0.3113, 0.2287],
        [0.0610, 0.4505, 0.3582,  ..., 0.3113, 0.0000, 0.0826],
        [0.1436, 0.5331, 0.4408,  ..., 0.2287, 0.0826, 0.0000]])

In [9]:
nodes_easy = torch.randint(low=0, high=10, size=(3, 1))
nodes_easy

tensor([[8],
        [0],
        [4]])

In [10]:
%timeit calculate_edges1(nodes_easy).reshape(3,3)

14.5 µs ± 3.82 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [11]:
nodes_easy_more = torch.stack([nodes_easy, nodes_easy]).permute(2, 0, 1)
easy_edges2 = calculate_edges2(nodes_easy_more)
easy_edges2_only = easy_edges2[0, 0]
antivectorized = anti_vectorize(easy_edges2_only, 3)

In [12]:
antivectorized

array([[0., 8., 4.],
       [8., 0., 4.],
       [4., 4., 0.]])

In [13]:
def calculate_adj(out, n_nodes):
    return torch.sum(
        torch.abs(
            out.repeat(n_nodes, 1, 1) - torch.transpose(out.repeat(n_nodes, 1, 1), 0, 1)
        ),
        2,
    )

In [14]:
def calculate_edges1_np(nodes, dim=1):
    return np.absolute(np.expand_dims(nodes, dim) - nodes)

In [15]:
torch.from_numpy(calculate_edges1_np(nodes_easy.numpy())).squeeze()

tensor([[0, 8, 4],
        [8, 0, 4],
        [4, 4, 0]])

In [16]:
nodes_easy.shape

torch.Size([3, 1])

In [17]:
nodes_complex = torch.randint(low=0, high=10, size=(1, 2, 5, 3))

In [18]:
pairwise = torch.from_numpy(
    calculate_edges1_np(nodes_complex.permute(0, 3, 1, 2).numpy(), dim=2)
)

In [19]:
nodes_complex[0, 0, 0]

tensor([8, 1, 2])

In [20]:
pairwise[0, :, :, 0, 0]

tensor([[0, 7, 6],
        [7, 0, 1],
        [6, 1, 0]])

In [21]:
%timeit calculate_adj(nodes_easy, 3)

41.4 µs ± 5.57 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
